In [33]:
from hypothesis_jsonschema import from_schema
from importlib import reload # For reload to work need to import the module level for a method
from pyspark.sql import SparkSession
from pyspark.sql import types as T

from ch7_testing import faker_example, util

In [3]:
spark = (SparkSession
         .builder
         .appName('oreilly-book')
         .getOrCreate())

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/07 15:17:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/07 15:17:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [22]:
reload(util)

<module 'ch7_testing.util' from '/Users/gizmo/dev/oreilly_dataeng_book/ch7_testing/util.py'>

In [25]:
data,_ = faker_example.create_mock_data(10)
df = util.df_from_list_dict(data, spark.sparkContext)

In [32]:
df.schema.jsonValue()

{'type': 'struct',
 'fields': [{'name': 'count',
   'type': 'long',
   'nullable': True,
   'metadata': {}},
  {'name': 'description', 'type': 'string', 'nullable': True, 'metadata': {}},
  {'name': 'img_files',
   'type': {'type': 'array', 'elementType': 'string', 'containsNull': True},
   'nullable': True,
   'metadata': {}},
  {'name': 'location',
   'type': {'type': 'struct',
    'fields': [{'name': '_1',
      'type': 'string',
      'nullable': True,
      'metadata': {}},
     {'name': '_2', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': '_3', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': '_4', 'type': 'string', 'nullable': True, 'metadata': {}},
     {'name': '_5', 'type': 'string', 'nullable': True, 'metadata': {}}]},
   'nullable': True,
   'metadata': {}},
  {'name': 'user', 'type': 'string', 'nullable': True, 'metadata': {}}]}

In [57]:
df.schema.fields

[StructField(count,LongType,true),
 StructField(description,StringType,true),
 StructField(img_files,ArrayType(StringType,true),true),
 StructField(location,StructType(List(StructField(_1,StringType,true),StructField(_2,StringType,true),StructField(_3,StringType,true),StructField(_4,StringType,true),StructField(_5,StringType,true))),true),
 StructField(user,StringType,true)]

In [46]:
# from_schema(df.schema.jsonValue()['fields'][0])
# df.schema.jsonValue()['fields'][0]
from_schema({"name": "count", "type": "integer", "nullable": True, "metadata": {}})

integers()

In [31]:
from_schema({"type": "integer", "minimum": 1, "exclusiveMaximum": 10})

integers(min_value=1, max_value=9)

In [50]:
# from_schema({'type': 'struct', 'fields': [{'name': 'count','type': 'integer','nullable': True,'metadata': {}}]})
from_schema({'fields': [{'name': 'count','type': 'integer','nullable': True,'metadata': {}}]})

builds(error_raiser)

In [58]:
test_schema = {
    "name": "user", "type": "text",
    "name": "location", "type": "text",
    "name": "description", "type": "text",
    "name": "count", "type": "integer"
    }

In [59]:
from_schema(test_schema)

integers()

22/05/07 20:06:22 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 415090 ms exceeds timeout 120000 ms
22/05/07 20:06:22 WARN SparkContext: Killing executors is not supported by current scheduler.
